In [ ]:
!pip uninstall -y nltk
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from nltk.stem import PorterStemmer

# Load CSV file (file name)
file_name = "/content/ESG_wordlist.xlsx - ESG-Wordlist.csv"
df = pd.read_csv(file_name)

# Initialize Porter Stemmer
stemmer = PorterStemmer()

# Assuming words are in the first column, apply stemming
df["Stemmed_Words"] = df.iloc[:, 0].astype(str).apply(stemmer.stem)

# Save output to a new CSV
output_file = "stemmed_words.csv"
df.to_csv(output_file, index=False)

print(f"Processed words saved to {output_file}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/ESG_wordlist.xlsx - ESG-Wordlist.csv'

In [ ]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlretrieve
from datetime import datetime as dt
import os, re
from pathlib import Path


import codecs
# f=codecs.open("./data/AAPL/20151028.htm", 'r')

data_format = '10-k'
DOMIN = 'https://www.sec.gov'



class TenKDownloader:
    def __init__(self, cik, start_date, end_date):
        if type(cik) == str:
            cik = [cik]
        elif type(cik) == list:
            for i, ele in enumerate(cik):
                assert type(ele)==str, f'cik at index {i} is not string: %s'%type(ele)
        else:
            raise TypeError('CIK should be string or list of string, input type is %s'%type(cik))
        self.CIK = cik
        self.start_date = dt.strptime(start_date,'%Y%m%d')
        self.end_date = dt.strptime(end_date,'%Y%m%d')
        self.all_url = {}
        self.cwd = os.getcwd()

    def download(self, target = './data', reset_flag=False):
        os.chdir(self.cwd)
        os.chdir(target)
        for c in self.CIK:
            try:
                if reset_flag:
                    result = self._search_each(c)
                else:
                    if c in self.all_url:
                        continue
                    else:
                        result = self._search_each(c)
            except ValueError as info:
                print(info)
                continue
            try:
                os.mkdir(c)
            except FileExistsError:
                pass
            os.chdir(f'./{c}')
            for each in result:
                print(f'Downloading {c}:{each[0]} {each[1]}')
                filename = each[0]+str(each[each[1].rfind('.'):])
                urlretrieve(each[1], filename)
                print('File saved in {}'.format(os.getcwd()+'\\'+filename))
            self.all_url[c] = result
            os.chdir('..')
        os.chdir('..')

    def _search_each(self, cik):
        assert cik in self.CIK, '%s is not in CIK list'%cik
        url = f'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=10-k&dateb=&owner=exclude&count=40'
        search_page = requests.get(url)
        assert search_page.status_code == 200, 'request code for search page: %s' % search_page.status_code
        search_head = BeautifulSoup(search_page.content, 'html.parser')
        search_result = search_head.select('table.tableFile2 tr')
        if len(search_result)==0:
            raise ValueError(f'Result for {cik} is not available, {url}')
        search_result.pop(0)
        start_idx, end_idx = self._search_date([self._get(item, 'date') for item in search_result], self.start_date, self.end_date)
        # print(start_idx, end_idx)
        result = []
        for i in range(start_idx, end_idx+1):
            if self._get(search_result[i], 'type')!='10-K':
                # print(self._get(search_result[i], 'date').strftime('%Y%m%d'))
                continue
            date = self._get(search_result[i], 'date').strftime('%Y%m%d')
            sub_url = DOMIN + search_result[i].find('a', attrs={"id": "documentsbutton"})['href']
            company_page = requests.get(sub_url)
            assert  company_page.status_code == 200, 'request code for company page: %s' % company_page.status_code
            company_head = BeautifulSoup(company_page.content, 'html.parser')
            file_table = company_head.select('table.tableFile')[0].select('tr')
            file_table.pop(0)
            for item in file_table:
                if '10-K' in item.select('td')[3].contents[0]:
                    break
            doc_url = item.select('td a')[0]['href']
            result.append((date, DOMIN+doc_url))
        return result

    def _get(self, item, info):
        if info == 'date':
            date = item.select('td')[3].contents[0]
            ret = dt.strptime(date,'%Y-%m-%d')
        elif info == 'url':
            ret = DOMIN + item.find('a', attrs={"id": "documentsbutton"})['href']
        elif info == 'type':
            ret = item.select('td')[0].contents[0]
        else:
            raise NotImplementedError
        return ret

    def _search_date(self, ls, start, end):
        h, t = ls[-1], ls[0]
        n = len(ls)
        assert start <= t and end >= h, f'Available time interval: {h} to {t}, input: {start} to {end}'
        # print(h,t)
        if start >= h:
            ei, _ = self._bsearch_dec(ls, start)
        else:
            ei = len(ls)-1
        if end <= t:
            _, si = self._bsearch_dec(ls, end)
        else:
            si = 0
        return si, ei

    def _bsearch_dec(self, ls, point):
        a = 0
        b = len(ls)
        while b-a > 1:
            tmp = int((a+b)/2)
            if ls[tmp] >= point:
                a = tmp
            else:
                b = tmp
        return a,b


class TenKScraper:
    def __init__(self, section, next_section):
        self.all_section = [str(i) for i in range(1, 16)] + ['1A', '1B', '7A', '9A', '9B']
        section = re.findall(r'\d.*\w*', section.upper())[0]
        next_section = re.findall(r'\d.*\w*', next_section.upper())[0]
        if section  not in self.all_section:
            raise ValueError(f'Section: {section} is not avaiable, avaiable section: {self.all_section}')
        if next_section  not in self.all_section:
            raise ValueError(f'Section: {next_section} is not avaiable, avaiable section: {self.all_section}')
        self.section = 'Item ' + section
        self.next_section = 'Item ' + next_section

    def scrape(self, input_path, output_path):
        with open(input_path, 'rb') as input_file:
            page = input_file.read()  # <===Read the HTML file into Python
            # Pre-processing the html content by removing extra white space and combining then into one line.
            page = page.strip()  # <=== remove white space at the beginning and end
            page = page.replace(b'\n', b' ')  # <===replace the \n (new line) character with space
            page = page.replace(b'\r', b'')  # <===replace the \r (carriage returns -if you're on windows) with space
            page = page.replace(b'&nbsp;',
                                b' ')  # <===replace "&nbsp;" (a special character for space in HTML) with space.
            page = page.replace(b'&#160;',
                                b' ')  # <===replace "&#160;" (a special character for space in HTML) with space.

            while b'  ' in page:
                page = page.replace(b'  ', b' ')  # <===remove extra space

            # Using regular expression to extract texts that match a pattern

            # Define pattern for regular expression.
            # The following patterns find ITEM 1 and ITEM 1A as diplayed as subtitles
            # (.+?) represents everything between the two subtitles
            # If you want to extract something else, here is what you should change

            # Define a list of potential patterns to find ITEM 1 and ITEM 1A as subtitles
            p1 = bytes(r'bold;\">\s*' + self.section + r'\.(.+?)bold;\">\s*' + self.next_section + r'\.',
                       encoding='utf-8')
            p2 = bytes(r'b>\s*' + self.section + r'\.(.+?)b>\s*' + self.next_section + r'\.', encoding='utf-8')
            p3 = bytes(r'' + self.section + r'\.\s*<\/b>(.+?)' + self.next_section + r'\.\s*<\/b>', encoding='utf-8')
            p4 = bytes(r'' + self.section + r'\.\s*[^<>]+\.\s*<\/b(.+?)' + self.next_section + r'\.\s*[^<>]+\.\s*<\/b',
                       encoding='utf-8')
            p5 = bytes(r'b>\s*<font[^>]+>\s*' + self.section + r'\.(.+?)b>\s*<font[^>]+>\s*' + self.next_section + r'\.', encoding='utf-8')
            p6 = bytes(r'' + self.section.upper() + r'\.\s*<\/b>(.+?)' + self.next_section.upper() + r'\.\s*<\/b>', encoding='utf-8')

            p7 = bytes(r'underline;\">\s*' + self.section + r'\<\/font>(.+?)underline;\">\s*'+ self.next_section + r'\.\s*\<\/font>',encoding = 'utf-8')
            p8 = bytes(r'underline;\">\s*' + self.section + r'\.\<\/font>(.+?)underline;\">\s*'+ self.next_section + r'\.\s*\<\/font>',encoding = 'utf-8')
            p9 = bytes(r'<font[^>]+>\s*' + self.section + r'\:(.+?)\<font[^>]+>\s*'+self.next_section + r'\:\s*',encoding = 'utf-8')
            p10 = bytes(r'<font[^>]+>\s*' + self.section + r'\.\<\/font>(.+?)\<font[^>]+>\s*' + self.next_section + r'\.',encoding = 'utf-8')
            p11 = bytes(r'' + self.section + r'\.(.+?)<font[^>]+>\s*' + self.next_section + r'\.\<\/font>',encoding = 'utf-8')
            p12 = bytes(r'b>\s*<font[^>]+>\s*' + self.section + r'(.+?)b>\s*<font[^>]+>\s*' + self.next_section + r'\s*\<\/font>', encoding='utf-8')
            p13 = bytes(r'' + self.section + r'\.\s*[^<>]+\.\s*<\/b(.+?)b>\s*' + self.next_section + r'\.',
                       encoding='utf-8')
            regexs = (
                p1,  # <===pattern 1: with an attribute bold before the item subtitle
                p2,  # <===pattern 2: with a tag <b> before the item subtitle
                p3,  # <===pattern 3: with a tag <\b> after the item subtitle
                p4,  # <===pattern 4: with a tag <\b> after the item+description subtitle
                p5,  # <===pattern 5: with a tag <b><font ...> before the item subtitle
                p6,  # <===pattern 6: with a tag <\b> after the item subtitle (ITEM XX.<\b>)
                p7,
                p8,
                p9,
                p10,
                p11,
                p12,
                p13)

            # Now we try to see if a match can be found...
            for regex in regexs:
                match = re.search(regex, page,
                                  flags=re.IGNORECASE)  # <===search for the pattern in HTML using re.search from the re package. Ignore cases.

                # If a match exist....
                if match:
                    # Now we have the extracted content still in an HTML format
                    # We now turn it into a beautiful soup object
                    # so that we can remove the html tags and only keep the texts

                    soup = BeautifulSoup(match.group(1),
                                         "html.parser")  # <=== match.group(1) returns the texts inside the parentheses (.*?)

                    # soup.text removes the html tags and only keep the texts
                    rawText = soup.text.encode('utf8')  # <=== you have to change the encoding the unicodes

                    # remove space at the beginning and end and the subtitle "business" at the beginning
                    # ^ matches the beginning of the text
                    # outText = re.sub(b"^business\s*", b"", rawText.strip(), flags=re.IGNORECASE)
                    Path(output_path).touch()
                    with open(output_path, "wb") as output_file:
                        output_file.write(rawText)

                    break  # <=== if a match is found, we break the for loop. Otherwise the for loop continues

        if match is None:
            print(f'No matched sections: {self.section}, {self.next_section} found in {input_path}.')
            return None
        else:
            return rawText

if __name__=='__main__':
    pass
    # print(os.getcwd())
    # os.chdir('./BIA660_project')
    # company_name = ['AAPL', 'GOOG']
    # downloader = TenKDownloader(company_name, '20150101','20180101')
    # downloader.download()

    # scraper = TenKScraper('Item 1A', 'Item 1B')  # scrape text start from Item 1A, and stop by Item 1B
    # scraper2 = TenKScraper('Item 7', 'Item 8')
    # scraper.scrape('./data/1326160/20110225.htm', './data/txt/test.txt')
    # scraper2.scrape('./data/1326160/20110225.htm', './data/txt/test2.txt')

In [ ]:
import sqlite3
import pandas as pd

# Connect to your database
conn = sqlite3.connect("database.sqlite")  # Replace with your actual database file
cursor = conn.cursor()

# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = cursor.fetchall()
# print(tables)
# Fetch company names and CIK numbers
cursor.execute("SELECT name, cik_key FROM companies")  # Replace 'your_table' with actual table name
data = cursor.fetchall()

# Convert to a DataFrame for easier handling
df = pd.DataFrame(data, columns=["name", "cik_key"])
print(df.head())  # Display first few rows


OperationalError: no such table: companies

In [ ]:
# Extract CIKs as a list of strings
cik_list = df["cik_key"].astype(str).tolist()
import os

# Define target directory
target_dir = "/content/data"  # Use absolute path in Colab

# Create the directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Now run the downloader with the correct path
downloader.download(target=target_dir, reset_flag=False)

# Set your desired date range
start_date = "20130101"  # Adjust to your required range
end_date = "20191231"

# Initialize and run the downloader
downloader = TenKDownloader(cik_list, start_date, end_date)
downloader.download(target="./data", reset_flag=False)


NameError: name 'df' is not defined

# New section

In [ ]:
from bs4 import BeautifulSoup
import re
import os
import glob

def clean_html(raw_html):
    """Remove HTML tags, attributes, and extra spaces from a document."""
    soup = BeautifulSoup(raw_html, "html.parser")
    text = soup.get_text(separator=" ")  # Extract text while preserving spaces
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Path to the folder containing HTML files
folder_path = "/content/data/"

# Get a list of up to 40 HTML files in the folder
html_files = glob.glob(os.path.join(folder_path, "*.htm"))[:40]  # Adjust if needed

# Process each file
cleaned_documents = {}

for file_path in html_files:
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()

    cleaned_text = clean_html(html_content)

    # Store the cleaned text in a dictionary
    cleaned_documents[file_path] = cleaned_text

    # Optionally, save cleaned text to new files
    output_file = file_path.replace(".htm", "_cleaned.txt")
    with open(output_file, "w", encoding="utf-8") as out_file:
        out_file.write(cleaned_text)

    print(f"Processed: {file_path}")

# Print a preview of the first cleaned document
print("\nSample cleaned text:\n", list(cleaned_documents.values())[0][:500])

Processed: /content/data/000149315222002707-form10-k.htm
Processed: /content/data/000106299322001764-form10ka.htm
Processed: /content/data/000000412722000002-swks-20211001.htm
Processed: /content/data/000001292722000010-ba-20211231.htm
Processed: /content/data/000155837022000635-gfe-20211031x10k.htm
Processed: /content/data/000121390022004518-f10kt2021_homebistro.htm
Processed: /content/data/000160706222000053-ibal063021formnt10k.htm
Processed: /content/data/000162828022001439-lwenergy-20210930.htm
Processed: /content/data/000143774922001938-bset20211127_10k.htm
Processed: /content/data/000110465922009075-arteu-20211231x10k.htm
Processed: /content/data/000173112222000120-e3461_nt10-k.htm
Processed: /content/data/000157570522000076-puge_12b25.htm
Processed: /content/data/000103544322000040-are-20211231.htm
Processed: /content/data/000121390022004524-f10k2021_european.htm
Processed: /content/data/000149315222002705-formnt10-k.htm
Processed: /content/data/000110465922009084-twnd-20211231x

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Download necessary nltk resources
nltk.download("punkt")

def load_dictionary(dictionary_path):
    """Load the 2of12inf dictionary into a set."""
    with open(dictionary_path, "r", encoding="utf-8") as file:
        valid_words = set(word.strip().lower() for word in file)
    return valid_words

def tokenize_and_filter(text, dictionary):
    """Tokenize text and filter words based on the 2of12inf dictionary."""
    tokens = word_tokenize(text)  # Tokenize text
    filtered_tokens = [word.lower() for word in tokens if word.lower() in dictionary]  # Keep only dictionary words
    return filtered_tokens

# Load dictionary
dictionary_path = "2of12inf.txt"  # Path to the 2of12inf dictionary file
valid_words = load_dictionary(dictionary_path)

# Example usage after cleaning HTML
tokenized_text = tokenize_and_filter(cleaned_text, valid_words)
print(tokenized_text[:50])  # Print first 50 tokens


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['true', 'no', 'true', 'shares', 'united', 'states', 'securities', 'and', 'exchange', 'commission', 'form', 'amendment', 'no', 'annual', 'report', 'pursuant', 'to', 'section', 'or', 'of', 'the', 'securities', 'exchange', 'act', 'of', 'for', 'the', 'fiscal', 'year', 'ended', 'or', 'transition', 'report', 'pursuant', 'to', 'section', 'or', 'of', 'the', 'securities', 'exchange', 'act', 'of', 'for', 'the', 'transition', 'period', 'from', 'commission', 'file']


In [ ]:
import nltk
from bs4 import BeautifulSoup
import re
import os
import glob

# Ensure required nltk resources are available
nltk.download('punkt')

def clean_html(raw_html):
    """Remove HTML tags, attributes, and extra spaces from a document."""
    soup = BeautifulSoup(raw_html, "html.parser")
    text = soup.get_text(separator=" ")  # Extract text while preserving spaces
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

def tokenize_and_filter(text, valid_words):
    """Tokenize the text and keep only words present in the valid_words dictionary."""
    tokens = nltk.word_tokenize(text.lower())  # Tokenize and convert to lowercase
    filtered_tokens = [word for word in tokens if word in valid_words]  # Filter words
    return filtered_tokens

# Load the 20of12inf dictionary
dict_path = "/content/2of12inf.txt"
with open(dict_path, "r", encoding="utf-8") as file:
    valid_words = set(word.strip().lower() for word in file.readlines())  # Load words into a set

# Path to the folder containing HTML files
folder_path = "/content/data/"

# Get a list of up to 40 HTML files in the folder
html_files = glob.glob(os.path.join(folder_path, "*.htm"))[:40]

# Process each file
filtered_documents = {}

for file_path in html_files:
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()

    cleaned_text = clean_html(html_content)  # Step 1: Clean HTML
    tokenized_text = tokenize_and_filter(cleaned_text, valid_words)  # Step 2: Tokenize & Filter

    # Store the filtered tokens
    filtered_documents[file_path] = tokenized_text

    # Save filtered tokens to new files
    output_file = file_path.replace(".htm", "_filtered.txt")
    with open(output_file, "w", encoding="utf-8") as out_file:
        out_file.write(" ".join(tokenized_text))  # Save as space-separated words

    print(f"Processed: {file_path}")

# Print a preview of the first filtered document
print("\nSample filtered tokens:\n", list(filtered_documents.values())[0][:50])  # Show first 50 tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processed: /content/data/000149315222002707-form10-k.htm
Processed: /content/data/000106299322001764-form10ka.htm
Processed: /content/data/000000412722000002-swks-20211001.htm
Processed: /content/data/000001292722000010-ba-20211231.htm
Processed: /content/data/000155837022000635-gfe-20211031x10k.htm
Processed: /content/data/000121390022004518-f10kt2021_homebistro.htm
Processed: /content/data/000160706222000053-ibal063021formnt10k.htm
Processed: /content/data/000162828022001439-lwenergy-20210930.htm
Processed: /content/data/000143774922001938-bset20211127_10k.htm
Processed: /content/data/000110465922009075-arteu-20211231x10k.htm
Processed: /content/data/000173112222000120-e3461_nt10-k.htm
Processed: /content/data/000157570522000076-puge_12b25.htm
Processed: /content/data/000103544322000040-are-20211231.htm
Processed: /content/data/000121390022004524-f10k2021_european.htm
Processed: /content/data/000149315222002705-formnt10-k.htm
Processed: /content/data/000110465922009084-twnd-20211231x

In [ ]:
from nltk.stem import PorterStemmer
import glob
import os

# Initialize the Porter Stemmer
stemmer = PorterStemmer()

def apply_stemming(words):
    """Apply Porter Stemming to a list of words."""
    return [stemmer.stem(word) for word in words]

# Path to filtered text files
filtered_files = glob.glob("/content/data/*_filtered.txt")

for file_path in filtered_files:
    # Read the filtered words
    with open(file_path, "r", encoding="utf-8") as file:
        words = file.read().split()  # Read as a list of words

    # Apply stemming
    stemmed_words = apply_stemming(words)

    # Save stemmed words to a new file
    output_file = file_path.replace("_filtered.txt", "_stemmed.txt")
    with open(output_file, "w", encoding="utf-8") as out_file:
        out_file.write(" ".join(stemmed_words))  # Save as space-separated words

    print(f"Stemmed file saved: {output_file}")

# Print a preview of the first stemmed document
sample_file = filtered_files[0].replace("_filtered.txt", "_stemmed.txt")
with open(sample_file, "r", encoding="utf-8") as file:
    print("\nSample Stemmed Words:\n", file.read().split()[:50])  # Show first 50 words


Stemmed file saved: /content/data/000121390022004524-f10k2021_european_stemmed.txt
Stemmed file saved: /content/data/000000412722000002-swks-20211001_stemmed.txt
Stemmed file saved: /content/data/000139390522000029-dusyf_nt10k_stemmed.txt
Stemmed file saved: /content/data/000160706222000053-ibal063021formnt10k_stemmed.txt
Stemmed file saved: /content/data/000101738622000039-pedro_2021oct31-nt10k_stemmed.txt
Stemmed file saved: /content/data/000169298122000002-form12b25_stemmed.txt
Stemmed file saved: /content/data/000149315222002765-form10-k_stemmed.txt
Stemmed file saved: /content/data/000110465922009084-twnd-20211231x10ka_stemmed.txt
Stemmed file saved: /content/data/000147793222000506-guskin_10k_stemmed.txt
Stemmed file saved: /content/data/000168316822000568-yuengling_i10k-103121_stemmed.txt
Stemmed file saved: /content/data/000110465922009075-arteu-20211231x10k_stemmed.txt
Stemmed file saved: /content/data/000121390022004518-f10kt2021_homebistro_stemmed.txt
Stemmed file saved: /co

In [ ]:
import pandas as pd
import glob
from collections import Counter

# Load ESG Dictionary (the stemmed words)
ws = pd.read_csv("stemmed_words.csv")
esg_words = set(ws['Stemmed_Words'])  # ESG words as a set

# Get list of stemmed text files
stemmed_files = glob.glob("/content/data/*_stemmed.txt")

# Initialize an empty DataFrame to hold the matrix
matrix = pd.DataFrame(columns=sorted(esg_words))  # Columns are ESG words

# Process each stemmed document
for file_path in stemmed_files:
    with open(file_path, "r", encoding="utf-8") as file:
        # Read the document and tokenize into words
        document_words = file.read().split()

        # Count the frequency of each word in the document
        word_counts = Counter(document_words)

        # Calculate the total word count (including ESG and non-ESG words)
        total_word_count = len(document_words)

    # Create a row for the current document with frequency of each ESG word
    row = {word: word_counts.get(word, 0) for word in esg_words}

    # Add the total word count to the row
    row['Total_Word_Count'] = total_word_count

    # Append the row to the matrix (DataFrame)
    matrix = pd.concat([matrix, pd.DataFrame([row])], ignore_index=True)

# Save matrix to a CSV file
matrix.to_csv("/content/data/word_matrix.csv", index=False)
print("Word matrix saved: /content/data/word_matrix.csv")

# Display the first 5 rows to confirm
print(matrix.head())


Word matrix saved: /content/data/word_matrix.csv
  agricultur air alcohol align announc appreci approv asc assess atmospher  \
0          0   0       0    18       4       2    105   0     19         0   
1          0   0       0     0       1       2     78   0     13         0   
2          0   0       0     0       0       0      0   0      0         0   
3          0   0       0     0       0       0      2   0      3         0   
4          0   0       0     0       2       4    118   0     11         0   

   ... wetland whistleblow wilder wildlif wind woman women workplac zone  \
0  ...       0           0      0       0    6     0     1        0    0   
1  ...       0           0      0       0    5     0     0        0    0   
2  ...       0           0      0       0    0     0     0        0    0   
3  ...       0           0      0       0    0     0     0        0    0   
4  ...       0           0      0       0   11     0     2        0    0   

  Total_Word_Count  
0   

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Load word matrix CSV (each row represents a report and each column is an ESG word frequency)
word_matrix = pd.read_csv('/content/data/word_matrix.csv')

# Load abnormal returns (for example, if these are stored in a list)
# Assuming you have 38 abnormal return values corresponding to the 38 reports
abnormal_returns = np.array([0.02, 0.03, -0.01, 0.05, 0.01, -0.02, 0.03, -0.04, 0.02, 0.01,
                             0.03, -0.01, 0.02, 0.04, -0.03, 0.01, 0.02, 0.03, -0.01, 0.02,
                             0.01, 0.03, 0.05, -0.02, 0.03, 0.04, -0.01, 0.02, 0.01, 0.03,
                             0.02, -0.01, 0.04, 0.01, 0.02, 0.03, 0.01, 0.02])  # Example values

# Ensure the number of abnormal returns matches the number of reports
len(abnormal_returns) == word_matrix.shape[0]

True

In [ ]:
# # Add intercept (constant term) to the feature matrix
# X = sm.add_constant(word_matrix)

# # Check the shape of X to ensure it matches the data
# print(X.shape)  # It should have 39 columns (1 intercept + number of ESG words)
# print(X.head)
# # Fit OLS regression model
# ols_model = sm.OLS(abnormal_returns, X)
# ols_results = ols_model.fit()

# # Output the optimized weights (coefficients)
# optimized_weights = ols_results.params
# print("Optimized Weights:", optimized_weights)


(38, 303)
<bound method NDFrame.head of     const  agricultur  air  alcohol  align  announc  appreci  approv  asc  \
0     1.0           0    0        0     18        4        2     105    0   
1     1.0           0    0        0      0        1        2      78    0   
2     1.0           0    0        0      0        0        0       0    0   
3     1.0           0    0        0      0        0        0       2    0   
4     1.0           0    0        0      0        2        4     118    0   
5     1.0           3    9        0      3       13        1      37    0   
6     1.0           0    0        0      5        0        0      41    0   
7     1.0           0    0        0      0        0        0       0    0   
8     1.0           0    0        0      3        7        1     111    0   
9     1.0           0    0        0      4        1        2       2    0   
10    1.0           1   24        0      3       14        1      27    0   
11    1.0           0    1        0 

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Step 1: Load the word frequency matrix (word_matrix.csv)
word_matrix = pd.read_csv('/content/data/word_matrix.csv')  # Assuming first column is report names

# Step 2: Set initial weights to 1 for each ESG word
word_weights = {word: 1 for word in word_matrix.columns if word != 'Total_Word_Count'}  # Exclude 'Total_Word_Count' column

# Step 3: Calculate the ESG Sentiment Score for each report
def calculate_esg_sentiment(word_matrix, word_weights):
    sentiment_scores = []
    for report, row in word_matrix.iterrows():
        # Get the frequency of each ESG word in the report (F_i,j)
        frequencies = row.drop('Total_Word_Count').values  # Exclude 'Total_Word_Count' column

        # Get the total number of words in the report (Total_Word_Count column)
        total_words = row['Total_Word_Count']  # Using the Total_Word_Count column

        # Calculate the weighted sum for the ESG sentiment score
        weighted_sum = 0
        for word, freq in zip(word_weights.keys(), frequencies):
            weighted_sum += word_weights[word] * freq

        # Calculate ESG sentiment score
        esg_sentiment = weighted_sum / total_words if total_words > 0 else 0
        sentiment_scores.append(esg_sentiment)

    # Return a DataFrame of sentiment scores for each report
    return pd.Series(sentiment_scores, index=word_matrix.index, name='ESG Sentiment Score')

# Step 4: Calculate ESG sentiment scores with initial weights
esg_scores_initial = calculate_esg_sentiment(word_matrix, word_weights)
print("Initial ESG Sentiment Scores:")
print(esg_scores_initial)

# Step 5: Now, perform OLS regression to refine word weights based on market reactions
# Assuming you have the abnormal returns (Δr_i) data for each report
# You can load the abnormal returns data as follows:
#abnormal_returns = pd.read_csv('abnormal_returns.csv', index_col=0)  # Assuming same index as word_matrix

# Ensure that the number of reports in abnormal_returns matches word_matrix
assert abnormal_returns.shape[0] == word_matrix.shape[0], "Mismatch in number of reports"

# Prepare the feature matrix (X) and target vector (y)
# X will be the word frequencies for each report, and y will be the abnormal returns
X = word_matrix.drop(columns=['Total_Word_Count']).values  # Exclude 'Total_Word_Count' column
y = abnormal_returns  # Directly flatten without using `.values`

# Add a constant to the X matrix for the intercept term in OLS
X_with_intercept = sm.add_constant(X)

# Step 6: Fit the OLS model to find the best word weights
ols_model = sm.OLS(y, X_with_intercept)
ols_results = ols_model.fit()

# Get the estimated coefficients (including the intercept)
estimated_weights = ols_results.params[1:]  # Exclude the intercept
word_weights_ols = {word: weight for word, weight in zip(word_matrix.columns[:-1], estimated_weights)}  # Exclude 'Total_Word_Count'

# Step 7: Calculate ESG sentiment scores again using the refined weights from OLS
esg_scores_refined = calculate_esg_sentiment(word_matrix, word_weights_ols)
print("Refined ESG Sentiment Scores (after OLS):")
print(esg_scores_refined)

# Optionally: Save the results to CSV
esg_scores_initial.to_csv('esg_sentiment_scores_initial.csv', header=True)
esg_scores_refined.to_csv('esg_sentiment_scores_refined.csv', header=True)



Initial ESG Sentiment Scores:
0     0.027111
1     0.027515
2     0.018109
3     0.031529
4     0.026525
5     0.031841
6     0.026810
7     0.022263
8     0.026763
9     0.031401
10    0.035621
11    0.030518
12    0.074166
13    0.034277
14    0.021484
15    0.046380
16    0.022871
17    0.038816
18    0.030326
19    0.018587
20    0.065803
21    0.018779
22    0.040209
23    0.016949
24    0.029164
25    0.047006
26    0.022018
27    0.032027
28    0.031594
29    0.020142
30    0.044592
31    0.017886
32    0.033342
33    0.031874
34    0.027736
35    0.032728
36    0.025292
37    0.053405
Name: ESG Sentiment Score, dtype: float64
Refined ESG Sentiment Scores (after OLS):
0     4.313949e-07
1     7.921981e-07
2    -1.003273e-06
3     5.533982e-06
4     2.782532e-07
5    -1.075580e-07
6     7.278542e-07
7    -5.658372e-05
8     4.566564e-07
9     6.166832e-07
10    7.197379e-07
11   -3.842684e-08
12    3.064441e-06
13    2.238463e-06
14   -4.003638e-05
15    3.076412e-06
16    9.3450